# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [94]:
import numpy as np
import pandas as pd
import xlsxwriter
import requests
from scipy import stats
import math

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [95]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [96]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'NMLL/OB LAE)G( QT ANGTAEEDASCSSRK', 'calculationPrice': 'close', 'open': 134.14, 'openTime': 1667772283209, 'openSource': 'icfafoli', 'close': 133.4, 'closeTime': 1615665510044, 'closeSource': 'ofcfilia', 'high': 138.01, 'highTime': 1687323172452, 'highSource': 'itmy ec1du5ir ln pedeae', 'low': 133.5, 'lowTime': 1610213602191, 'lowSource': 't eepcaym1ndirde5 ulei ', 'latestPrice': 131.99, 'latestSource': 'Close', 'latestTime': 'December 24, 2020', 'latestUpdate': 1617523626000, 'latestVolume': 56598362, 'iexRealtimePrice': None, 'iexRealtimeSize': None, 'iexLastUpdated': None, 'delayedPrice': 138.28, 'delayedPriceTime': 1659007568825, 'oddLotDelayedPrice': 132.38, 'oddLotDelayedPriceTime': 1650463326568, 'extendedPrice': 137, 'extendedChange': 0.03, 'extendedChangePercent': 0.00023, 'extendedPriceTime': 1679338111469, 'previousClose': 131.24, 'previousVolume': 89412993, 'change': 1.05, 'changePercent': 0.00774, 'volume'

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [97]:
price = data['latestPrice']
pe_ratio = data['peRatio']
print(price, pe_ratio)

131.99 41.94


## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [98]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [99]:
final_dataframe = pd.DataFrame(columns=my_columns)
for symbol_string in symbol_strings:
    batch_api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['peRatio'],
            'N/A'
        ],
        index=my_columns),
        ignore_index=True) 
final_dataframe    

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,121.21,55.43,N/A
1,AAL,15.98,-1.16,N/A
2,AAP,161.68,23.6,N/A
3,AAPL,137.21,41.54,N/A
4,ABBV,105.68,23.14,N/A
...,...,...,...,...
500,YUM,108.63,32.4,N/A
501,ZBH,153.62,-212.4,N/A
502,ZBRA,394.19,45.11,N/A
503,ZION,44.41,18.48,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [100]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = True, inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0] 
final_dataframe = final_dataframe[:50]

In [101]:
final_dataframe.reset_index(inplace=True)
final_dataframe

,index,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,340,NRG,35.72,2.2,N/A
1,335,NLOK,21.91,4.06,N/A
2,459,UNM,21.99,4.71,N/A
3,19,AIV,5.17,4.99,N/A
4,63,BIO,600.18,5.16,N/A
5,265,KIM,14.83,6.7,N/A
6,304,MET,47.30,7.61,N/A
7,26,ALL,111.59,7.61,N/A
8,152,EBAY,50.90,7.91,N/A
9,62,BIIB,250.39,8.32,N/A


In [102]:
final_dataframe.drop('index', axis=1, inplace=True)


In [103]:
final_dataframe.columns

Index(['Ticker', 'Price', 'Price-to-Earnings Ratio',
       'Number of Shares to Buy'],
      dtype='object')

## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [104]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [105]:
portfolio_input()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [106]:
position_size = float(portfolio_size) / len(final_dataframe.index) 
for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,35.72,2.2,5599
1,NLOK,21.91,4.06,9128
2,UNM,21.99,4.71,9095
3,AIV,5.17,4.99,38684
4,BIO,600.18,5.16,333
5,KIM,14.83,6.7,13486
6,MET,47.30,7.61,4228
7,ALL,111.59,7.61,1792
8,EBAY,50.90,7.91,3929
9,BIIB,250.39,8.32,798


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [107]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()
print(data[symbol]['advanced-stats'])
# Price-to-earnings ratio
pe_ratio = data[symbol]['quote']['peRatio']

# Price-to-book ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# Price-to-sales ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
enterprice_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprice_value / ebitda

# Enterprise Value divided by Gross Profit (EV/GP)
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit= enterprice_value / gross_profit
print(pe_ratio, pb_ratio, ps_ratio, ev_to_ebitda, ev_to_gross_profit)

{'beta': 1.160149062592319, 'totalCash': 92189925967, 'currentDebt': 105594254458, 'revenue': 285839026840, 'grossProfit': 106947478791, 'totalRevenue': 283278976456, 'EBITDA': 77808234343, 'revenuePerShare': 16.23, 'revenuePerEmployee': 2021169.81, 'debtToEquity': 5.182388240238173, 'profitMargin': 0.2174648958804959, 'enterpriseValue': 2367935576021, 'enterpriseValueToRevenue': 8.53, 'priceToSales': 8.18, 'priceToBook': 35.93, 'forwardPERatio': 34.62932519896053, 'pegRatio': 3.95330779082331, 'peHigh': 42.630501892079835, 'peLow': 16.819673988023897, 'week52highDate': '2020-08-31', 'week52lowDate': '2020-03-13', 'putCallRatio': 0.32246308903684806, 'companyName': 'Apple Inc', 'marketcap': 2350572194162, 'week52high': 134.08, 'week52low': 57.07, 'week52change': 0.8755888973019854, 'sharesOutstanding': 17282078095, 'float': None, 'avg10Volume': 126595606, 'avg30Volume': 112754457, 'day200MovingAvg': 113.81, 'day50MovingAvg': 124.79, 'employees': None, 'ttmEPS': 3.35, 'ttmDividendRate':

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [108]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)
for symbol_string in symbol_strings[:1]:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprice_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        try:
            ev_to_ebitda = enterprice_value / ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_gross_profit = enterprice_value / gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'                
                
                
        ], index=rv_columns),
        ignore_index=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.68,N/A,54.45,N/A,7.38,N/A,6.89,N/A,34.809297,N/A,13.577157,N/A,N/A
1,AAL,16.43,N/A,-1.16,N/A,-1.74,N/A,0.4023,N/A,-6.095354,N/A,1.382997,N/A,N/A
2,AAP,161.59,N/A,23.77,N/A,3,N/A,1.15,N/A,10.776207,N/A,2.498313,N/A,N/A
3,AAPL,134.08,N/A,41.14,N/A,35.83,N/A,8.53,N/A,29.611228,N/A,21.866260,N/A,N/A
4,ABBV,103.67,N/A,22.81,N/A,12.37,N/A,4.6,N/A,15.622256,N/A,9.528732,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,87.94,N/A,18.2,N/A,1.45,N/A,1.88,N/A,8.294281,N/A,1.809419,N/A,N/A
96,CL,86.72,N/A,27.39,N/A,112.73,N/A,4.7,N/A,20.344691,N/A,8.124690,N/A,N/A
97,CLX,212.50,N/A,23.56,N/A,23.77,N/A,3.8,N/A,19.061357,N/A,8.203548,N/A,N/A
98,CMA,57.90,N/A,15.39,N/A,1.07,N/A,2.48,N/A,10.237838,N/A,2.869650,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [109]:
colums_avg = ['Price-to-Earnings Ratio', 'Price-to-Book Ratio', 'Price-to-Sales Ratio', 'EV/EBITDA', 'EV/GP']
for column in colums_avg:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.68,N/A,54.45,N/A,7.38,N/A,6.8900,N/A,34.809297,N/A,13.577157,N/A,N/A
1,AAL,16.43,N/A,-1.16,N/A,-1.74,N/A,0.4023,N/A,-6.095354,N/A,1.382997,N/A,N/A
2,AAP,161.59,N/A,23.77,N/A,3.00,N/A,1.1500,N/A,10.776207,N/A,2.498313,N/A,N/A
3,AAPL,134.08,N/A,41.14,N/A,35.83,N/A,8.5300,N/A,29.611228,N/A,21.866260,N/A,N/A
4,ABBV,103.67,N/A,22.81,N/A,12.37,N/A,4.6000,N/A,15.622256,N/A,9.528732,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,87.94,N/A,18.20,N/A,1.45,N/A,1.8800,N/A,8.294281,N/A,1.809419,N/A,N/A
96,CL,86.72,N/A,27.39,N/A,112.73,N/A,4.7000,N/A,20.344691,N/A,8.124690,N/A,N/A
97,CLX,212.50,N/A,23.56,N/A,23.77,N/A,3.8000,N/A,19.061357,N/A,8.203548,N/A,N/A
98,CMA,57.90,N/A,15.39,N/A,1.07,N/A,2.4800,N/A,10.237838,N/A,2.869650,N/A,N/A


In [110]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [111]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.68,N/A,54.45,87,7.38,70,6.8900,78,34.809297,88,13.577157,79,N/A
1,AAL,16.43,N/A,-1.16,8,-1.74,4,0.4023,3,-6.095354,3,1.382997,7,N/A
2,AAP,161.59,N/A,23.77,47,3.00,36,1.1500,19,10.776207,25,2.498313,17,N/A
3,AAPL,134.08,N/A,41.14,78,35.83,98,8.5300,84,29.611228,84,21.866260,95,N/A
4,ABBV,103.67,N/A,22.81,44,12.37,82,4.6000,61,15.622256,49,9.528732,62,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,87.94,N/A,18.20,29,1.45,19,1.8800,26,8.294281,16,1.809419,10,N/A
96,CL,86.72,N/A,27.39,54,112.73,99,4.7000,64,20.344691,68,8.124690,53,N/A
97,CLX,212.50,N/A,23.56,46,23.77,95,3.8000,53,19.061357,57,8.203548,54,N/A
98,CMA,57.90,N/A,15.39,23,1.07,11,2.4800,40,10.237838,24,2.869650,21,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [112]:
from statistics import mean
for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,120.68,N/A,54.45,87,7.38,70,6.8900,78,34.809297,88,13.577157,79,80.4
1,AAL,16.43,N/A,-1.16,8,-1.74,4,0.4023,3,-6.095354,3,1.382997,7,5
2,AAP,161.59,N/A,23.77,47,3.00,36,1.1500,19,10.776207,25,2.498313,17,28.8
3,AAPL,134.08,N/A,41.14,78,35.83,98,8.5300,84,29.611228,84,21.866260,95,87.8
4,ABBV,103.67,N/A,22.81,44,12.37,82,4.6000,61,15.622256,49,9.528732,62,59.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,CINF,87.94,N/A,18.20,29,1.45,19,1.8800,26,8.294281,16,1.809419,10,20
96,CL,86.72,N/A,27.39,54,112.73,99,4.7000,64,20.344691,68,8.124690,53,67.6
97,CLX,212.50,N/A,23.56,46,23.77,95,3.8000,53,19.061357,57,8.203548,54,61
98,CMA,57.90,N/A,15.39,23,1.07,11,2.4800,40,10.237838,24,2.869650,21,23.8


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [113]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,16.43,N/A,-1.16,8,-1.7400,4,0.4023,3,-6.095354,3,1.382997,7,5
1,AIG,37.86,N/A,-6.84,4,0.5270,6,0.7074,8,4.174491,6,0.710590,2,5.2
2,BA,222.20,N/A,-27.60,3,-10.4600,2,2.1000,33,-29.334040,1,-208.158877,1,8
3,ALL,109.26,N/A,7.86,12,1.3100,16,0.7395,9,2.471202,5,0.753739,4,9.2
4,APA,14.70,N/A,-0.71,9,-3.4500,3,1.0800,17,7.402673,14,2.682193,18,12.2
5,AFL,44.81,N/A,11.47,15,1.0600,10,1.4000,21,5.945364,7,1.388650,8,12.2
6,C,62.66,N/A,11.89,16,0.7268,7,1.3700,20,6.899777,11,1.599170,9,12.6
7,AIZ,137.07,N/A,20.25,37,1.3700,17,0.7665,10,1.804161,4,0.733040,3,14.2
8,BK,41.55,N/A,9.40,14,0.9180,9,2.0400,32,6.722862,10,2.017434,13,15.6
9,AIV,5.22,N/A,5.06,11,0.3810,5,0.8742,12,9.126202,19,5.634966,33,16


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [114]:
portfolio_input()

Enter the value of your portfolio:2500000


In [115]:
position_size = float(portfolio_size) / len(rv_dataframe.index)

for row in rv_dataframe.index:
    # print(position_size / rv_dataframe.loc[row, 'Price'])
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe.loc[row, 'Price'])
rv_dataframe

/Users/mb/Documents/algorithmic-trading-python/venv/lib/python3.8/site-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,16.43,3043,-1.16,8,-1.7400,4,0.4023,3,-6.095354,3,1.382997,7,5
1,AIG,37.86,1320,-6.84,4,0.5270,6,0.7074,8,4.174491,6,0.710590,2,5.2
2,BA,222.20,225,-27.60,3,-10.4600,2,2.1000,33,-29.334040,1,-208.158877,1,8
3,ALL,109.26,457,7.86,12,1.3100,16,0.7395,9,2.471202,5,0.753739,4,9.2
4,APA,14.70,3401,-0.71,9,-3.4500,3,1.0800,17,7.402673,14,2.682193,18,12.2
5,AFL,44.81,1115,11.47,15,1.0600,10,1.4000,21,5.945364,7,1.388650,8,12.2
6,C,62.66,797,11.89,16,0.7268,7,1.3700,20,6.899777,11,1.599170,9,12.6
7,AIZ,137.07,364,20.25,37,1.3700,17,0.7665,10,1.804161,4,0.733040,3,14.2
8,BK,41.55,1203,9.40,14,0.9180,9,2.0400,32,6.722862,10,2.017434,13,15.6
9,AIV,5.22,9578,5.06,11,0.3810,5,0.8742,12,9.126202,19,5.634966,33,16


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [140]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

# writer = pd.ExcelWriter('value_strategy.xlsx', engine = 'xlsxwriter')
# rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [141]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0.0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [142]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }
for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [143]:
writer.save()